In [1]:
!pip install stable-baselines3[extra]

In [3]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...


In [5]:
import requests
import sys
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import torch

In [7]:
sys.path.insert(0,'boptestGymService')

url = 'https://api.boptest.net'

In [9]:
from boptestGymService.boptestGymEnv import BoptestGymEnv
from boptestGymEnv import DiscretizedActionWrapper

C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:141: SyntaxWarning: invalid escape sequence '\d'
  axs[3].plot([],[], color='gold',        linestyle='-', linewidth=1, label='$\dot{Q}_{rad}$')


In [11]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    '''Define a custom reward for this building

    '''
    def get_reward(self):
        '''Custom reward function. To expedite learning, we use a clipped reward
        function that has a value of 1 when there is no increase in discomfort
        and 0 otherwise. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

In [13]:
BoptestGymEnv?


Init signature:
BoptestGymEnv(
    url='http://127.0.0.1',
    testcase='bestest_hydronic_heat_pump',
    actions=['oveHeaPumY_u'],
    observations={'reaTZon_y': (280.0, 310.0)},
    reward=['reward'],
    max_episode_length=10800,
    random_start_time=False,
    excluding_periods=None,
    regressive_period=None,
    predictive_period=None,
    start_time=0,
    warmup_period=0,
    scenario={'electricity_price': 'constant'},
    step_period=900,
    render_episodes=False,
    log_dir='C:\\Users\\irmak\\Documents\\GitHub\\RL_ex',
)
Docstring:     
BOPTEST Environment that follows gym interface.
This environment allows the interaction of RL agents with building
emulator models from BOPTEST. 
 
Init docstring:
Parameters
----------
url: string
    Rest API url for communication with the BOPTEST interface
testcase: string
    The string identifier of the testcase
actions: list
    List of strings indicating the action space. The bounds of 
    each variable from the action space the ar

In [67]:
    import logging

    def pretty_print(title, data):
        logging.info(f"\n--- {title} ---\n{json.dumps(data, indent=2)}")

    def celsius_to_kelvin(c):
        return c + 273.15

# Configure logger
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

    logging.info("Connecting to BOPTEST...")

    # 2. Select test case
    resp = requests.post(f"{url}/testcases/{test_case}/select")
    if resp.status_code != 200:
        logging.error(f"Error selecting test case: {resp.status_code} {resp.text}")
    testid = resp.json()["testid"]
    logging.info(f"Selected test case ID: {testid}")

    # 3. Initialize simulation
    resp = requests.put(
        f"{url}/initialize/{testid}",
        json={"start_time": 0, "warmup_period": 0,
            }
    )
    if resp.status_code != 200:
        logging.error(f"Error initializing simulation: {resp.status_code} {resp.text}")
    logging.info("Initialization completed")

    # 4. Set step time to 300 seconds (5 minutes)
    resp = requests.put(f"{url}/step/{testid}", json={"step": 300})
    if resp.status_code != 200:
        logging.error(f"Error setting simulation step: {resp.status_code} {resp.text}")
    else:
        logging.info("Step time set to 300 seconds")

    # 5. Get measurement signals
    resp = requests.get(f"{url}/measurements/{testid}")
    if resp.status_code != 200:
        logging.error(f"Error fetching measurements: {resp.status_code} {resp.text}")
    pretty_print("Measurement Signals", resp.json())

    # 6. Get input signals
    resp = requests.get(f"{url}/inputs/{testid}")
    if resp.status_code != 200:
        logging.error(f"Error fetching control signals: {resp.status_code} {resp.text}")
    inputs = resp.json()["payload"]
    pretty_print("Control Signals", inputs)

    # 11. Stop test
    resp = requests.put(f"{url}/stop/{testid}")
    if resp.status_code != 200:
        logging.error(f"Error stopping test: {resp.status_code} {resp.text}")
    logging.info("Test successfully stopped")

2025-05-06 15:58:12,136 - INFO - Connecting to BOPTEST...
2025-05-06 15:58:16,857 - INFO - Selected test case ID: ebe66fc7-c3ad-422b-a30c-0048bc8f1b23
2025-05-06 15:58:17,752 - INFO - Initialization completed
2025-05-06 15:58:18,452 - INFO - Step time set to 300 seconds
2025-05-06 15:58:19,176 - INFO - 
--- Measurement Signals ---
{
  "status": 200,
  "message": "Queried the measurements successfully.",
  "payload": {
    "chi_reaFloSup_y": {
      "Unit": "m3/s",
      "Description": "Supply water flow rate of chiller",
      "Minimum": null,
      "Maximum": null
    },
    "chi_reaPChi_y": {
      "Unit": "W",
      "Description": "Electric power consumed by chiller",
      "Minimum": null,
      "Maximum": null
    },
    "chi_reaPPumDis_y": {
      "Unit": "W",
      "Description": "Electric power consumed by chilled water distribution pump",
      "Minimum": null,
      "Maximum": null
    },
    "chi_reaTRet_y": {
      "Unit": "K",
      "Description": "Return water temperature

In [19]:
test_case="multizone_office_simple_air"

In [65]:
env = BoptestGymEnv(url                   = url,
                    testcase              = test_case,
                    actions               = ['hvac_oveAhu_TSupSet_u'],
                    observations          = {"relHum":(0., 100.)},
                    random_start_time     = False,
                    start_time            = 31*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 0,
                    step_period           = 3600,
                   predictive_period=10)

In [8]:
env = DiscretizedActionWrapper(env,n_bins_act=1)

In [9]:
log_path =os.path.join("Training", "Boptest", "Logs")
PPO_path = os.path.join("Training" , "Saved Models", "PPO_Boptest")

In [10]:
PPO?

Init signature:
PPO(
    policy: Union[str, type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]]

In [11]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, device="cpu", learning_rate=0.001, gamma= 0.99, batch_size= 64)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=10)

Logging to Training\Boptest\Logs\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24       |
|    ep_rew_mean     | -2.06    |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 3303     |
|    total_timesteps | 2048     |
---------------------------------


In [13]:
#model.save(PPO_path)

In [14]:
#del(model)

In [15]:
#model = PPO.load('PPO_Boptest', env=env, device="cpu")

In [16]:
evaluate_policy(model, env, n_eval_episodes=3)

C:\Users\irmak\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-12.053104348946363, 0.0)

In [17]:
from boptestGymEnv import DiscretizedObservationWrapper

In [18]:
lower_setp = 21 + 273.15
upper_setp = 24 + 273.15

In [19]:
import gymnasium as gym
from gym.wrappers import FlattenObservation

In [37]:
print('Observation space of the building environment:')
print(env.observation_space.sample())
print(env.observation_space)
print('Action space of the building environment:')
print(env.action_space)
print(env.action_space.sample())

Observation space of the building environment:
2
Discrete(3)
Action space of the building environment:
Discrete(2)
0


In [39]:
env = BoptestGymEnvCustomReward(url                   = url,
                                testcase              = 'bestest_hydronic_heat_pump',
                                actions               = ['oveHeaPumY_u'],
                                observations          = {'reaTZon_y':(lower_setp,upper_setp)},
                                random_start_time     = False,
                                start_time            = 31*24*3600,
                                max_episode_length    = 24*3600,
                                warmup_period         = 24*3600,
                                step_period           = 3600)
env = DiscretizedActionWrapper(env, n_bins_act=1)
env = DiscretizedObservationWrapper(env, n_bins_obs=3, outs_are_bins=True)

done = False
obs, _ = env.reset()

from IPython.display import clear_output
while not done:
  # Clear the display output at each step
  clear_output(wait=True)
  # Compute control signal
  action, _states = model.predict(obs, deterministic=True)
  # Print the current operative temperature and decided action
  print('-------------------------------------------------------------------')
  print('State  [Bin #]  = {}'.format(obs[0]-273.15))
  print('Action [ - ]    = {:.0f}'.format(action))
  print('-------------------------------------------------------------------')
  # Implement action
  obs,reward,terminated,truncated,info = env.step(action) # send the action to the environment
  done = (terminated or truncated)

ValueError: Error: Unexpected observation shape () for Box environment, please use (1,) or (n_env, 1) for the observation shape.